In [ ]:
!pip install mtranslate

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import time
import sys
import pandas as pd
import numpy as np
import re
import datetime

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def is_valid_url(url):
    """Validate the URL to ensure it's in the correct format."""
    return url.startswith("https://")

def fetch_article_urls(base_url, num_pages=1):
    """Scrape article URLs from the Hespress website."""
    article_urls = []
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
        )
    }
    session = requests.Session()
    session.headers.update(headers)

    for page in range(1, num_pages + 1):
        url = f"{base_url}/page/{page}" if page > 1 else base_url
        try:
            response = session.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, "html.parser")
                links = soup.find_all("a", class_="stretched-link")
                for link in links:
                    href = link.get("href")
                    if href and is_valid_url(href):
                        article_urls.append(href)
            else:
                print(f"Failed to fetch page {page} with status code {response.status_code}")
        except Exception as e:
            print(f"An error occurred while fetching page {page}: {e}")

        # Add a delay between requests to avoid being blocked
        time.sleep(2)

    return list(set(article_urls))  # Remove duplicates

# Replace 'https://www.hespress.com' with the base URL of the website
base_url = "https://www.hespress.com/politique"
num_pages = 5  # Number of pages to scrape

article_urls = fetch_article_urls(base_url, num_pages)

if article_urls:
    print(f"Found {len(article_urls)} articles.")
    for url in article_urls:
        print(url)
else:
    print("No articles found.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def is_valid_url(url):
    """Validate the URL to ensure it's in the correct format."""
    return url.startswith("https://")

def fetch_article_urls(base_url, num_pages=1):
    """Scrape article URLs from the Hespress website."""
    article_urls = []
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
        )
    }
    session = requests.Session()
    session.headers.update(headers)

    for page in range(1, num_pages + 1):
        url = f"{base_url}/page/{page}" if page > 1 else base_url
        try:
            response = session.get(url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, "html.parser")
                links = soup.find_all("a", class_="stretched-link")
                for link in links:
                    href = link.get("href")
                    if href and is_valid_url(href):
                        article_urls.append(href)
            else:
                print(f"Failed to fetch page {page} with status code {response.status_code}")
        except Exception as e:
            print(f"An error occurred while fetching page {page}: {e}")

        # Add a delay between requests to avoid being blocked
        time.sleep(2)

    return list(set(article_urls))  # Remove duplicates

# Replace 'https://www.hespress.com/politique' with the base URL of the website
base_url = "https://www.hespress.com/politique"
num_pages = 5  # Number of pages to scrape

article_urls = fetch_article_urls(base_url, num_pages)

# Print the list of URLs
if article_urls:
    print(f"Found {len(article_urls)} articles.")
    for url in article_urls:
        print(url)
else:
    print("No articles found.")


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import requests
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import string
import nltk
from mtranslate import translate
import seaborn as sns
import matplotlib.pyplot as plt

# Download NLTK resources
nltk.download('stopwords')
nltk.download('vader_lexicon')

def is_valid_url(url):
    try:
        result = urlparse(url)
        if all([result.scheme, result.netloc]) and result.netloc == "www.hespress.com":
            return True
        return False
    except ValueError:
        return False

def arabic_to_english_month(arabic_month):
    months_arabic = ['يناير', 'فبراير', 'مارس', 'أبريل', 'ماي', 'يونيو', 'يوليوز', 'غشت', 'شتنبر', 'أكتوبر', 'نونبر', 'دجنبر']
    months_english = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    return months_english[months_arabic.index(arabic_month)]

def fetch_comments(url):
    user_name = []
    comment_text = []
    comment_date = []
    likes = []

    headers = {'User-Agent': 'Mozilla/5.0'}
    data = requests.get(url, headers=headers)
    soup = BeautifulSoup(data.content, "html.parser")

    article_title_tag = soup.find('h1', class_='post-title')
    article_title = article_title_tag.get_text() if article_title_tag else 'Unknown Title'

    c = soup.find('ul', {"class": "comment-list hide-comments"})

    if c:
        for li in c.find_all('li', class_='comment'):
            user_span = li.find('span', class_='fn heey')
            user_name.append(user_span.get_text() if user_span else 'Unknown')

            comment_p = li.find('p')
            comment_text.append(comment_p.get_text() if comment_p else 'No comment text found')

            date_span = li.find('div', class_='comment-date')
            if date_span:
                date_string = date_span.get_text().strip()
                date_parts = date_string.split()
                day = int(date_parts[1])
                month_arabic = date_parts[2]
                month = arabic_to_english_month(month_arabic)
                year = int(date_parts[3])
                time_parts = date_parts[-1].split(':')
                hour = int(time_parts[0])
                minute = int(time_parts[1])
                comment_date.append(pd.Timestamp(year, datetime.strptime(month, '%B').month, day, hour, minute))
            else:
                comment_date.append('Unknown date')

            likes_span = li.find('span', class_='comment-recat-number')
            likes.append(int(likes_span.get_text()) if likes_span else 0)
    else:
        print(f"No comments found for URL: {url}")
        return pd.DataFrame(columns=['User Name', 'Comment', 'Date', 'Likes']), article_title

    df = pd.DataFrame({'User Name': user_name, 'Comment': comment_text, 'Date': comment_date, 'Likes': likes})
    return df, article_title

def analyze_sentiment(comment):
    sid = SentimentIntensityAnalyzer()
    arabic_stopwords = set(stopwords.words('arabic'))
    arabic_punctuation = set(string.punctuation)

    try:
        translated_comment = translate(comment, 'en')
        tokens = translated_comment.split()
        filtered_tokens = [word for word in tokens if word.lower() not in arabic_stopwords and word not in arabic_punctuation]
        preprocessed_comment = ' '.join(filtered_tokens)
        scores = sid.polarity_scores(preprocessed_comment)
        compound_score = scores['compound']
    except Exception as e:
        print(f"Error occurred during sentiment analysis: {e}")
        return 'Neutral', 0.0

    if compound_score >= 0.1:
        return 'Positive', compound_score
    elif compound_score <= -0.1:
        return 'Negative', compound_score
    else:
        return 'Neutral', compound_score

def create_bar_plot(comments_df):
    sentiment_counts = comments_df['Sentiment'].value_counts()
    plt.figure(figsize=(8, 6))
    sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values)
    plt.title('Sentiment Distribution')
    plt.xlabel('Sentiment')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('static/images/sentiment_distribution.png')

def create_time_series_plot(comments_df):
    comments_df['Date'] = pd.to_datetime(comments_df['Date'], errors='coerce')
    comments_df = comments_df.dropna(subset=['Date'])
    comments_df.set_index('Date', inplace=True)
    comments_over_time = comments_df.resample('D').size()
    plt.figure(figsize=(10, 6))
    comments_over_time.plot()
    plt.title('Number of Comments Over Time')
    plt.xlabel('Date')
    plt.ylabel('Number of Comments')
    plt.tight_layout()
    plt.savefig('static/images/comments_over_time.png')

if __name__ == "__main__":
    urls = article_urls


    combined_df = pd.DataFrame()

    for url in urls:
        if is_valid_url(url):
            comments_df, article_title = fetch_comments(url)
            if not comments_df.empty:
                comments_df['Sentiment'], comments_df['Sentiment Score'] = zip(*comments_df['Comment'].apply(analyze_sentiment))
                combined_df = pd.concat([combined_df, comments_df])
                print(f"Comments for '{article_title}' have been fetched and analyzed.")
            else:
                print(f"No comments found for the article: {article_title}")
        else:
            print(f"Invalid URL: {url}")

    if not combined_df.empty:
        combined_df.to_csv('comments_combined.csv', index=False)
        print("All comments have been saved to 'comments_combined.csv'")
        create_bar_plot(combined_df)
        create_time_series_plot(combined_df)
    else:
        print("No valid comments to analyze.")